# 多动作多智能体实例-狼人杀

这节课来学习Metagpt的一个多动作多智能体的实战案例-狼人杀游戏。

**游戏规则**

狼人杀是一款多人参与的社交推理游戏，游戏中的角色分为狼人、村民和特殊角色三大类。基本规则如下：

1. **角色分配**：游戏开始前，每位玩家随机分配一个角色，包括狼人、普通村民和具有特殊能力的神职村民（如预言家、女巫、猎人等）。
2. **游戏流程**：游戏分为夜晚和白天两个阶段。夜晚，狼人睁眼并杀害一名玩家；白天，所有玩家讨论并投票处决一名玩家。这个过程会不断重复，直到满足某个胜利条件。
3.   **胜利条件**：游戏的胜利条件分为狼人阵营胜利和村民阵营胜利。
**狼人胜利**：狼人数量等于村民数量时，狼人阵营获胜
**村民胜利**：所有狼人被找出并处决，村民阵营获胜

Metagpt多智能体代码核心关注三部分：

- **角色（Role）-智能体的角色**
- **动作（Action）-角色对应的动作**
- **交互环境（Environment）-串联各角色的消息实现智能体间的信息交互**

**定义角色**

1.角色包括：村民、狼人、守卫、先知、巫师、主持人

2.角色框架 - BasePlayer，该类封装了角色的基本行为和属性，所有的角色都继承自这个类，从这个类中派生。其基本属性和初始化如下：

- 首先角色都需要监听 InstructSpeak 动作产生的消息：self.\_watch([InstructSpeak])
- 角色的行为设置：self.set\_actions(capable\_actions)，包括设置进来的 special\_actions 和 Speak Action。

**定义动作**

主持人 Moderator 的主要职责是：开始游戏、主持流程、解析角色发言和宣布游戏结果。

村民继承自 BasePlayer，其拥有 Speak 行为。

狼人除了能 Speak （继承自 BasePlayer）外，拥有特殊技能 Hunt。狼人在白天时，要伪装成好人说话，所以，还有个额外的Action：Impersonate。狼人就两个动作：一个是夜晚干人，二是白天伪装成好人发言。

守卫的特殊技能：Protect，保护人。

先知的特殊技能：Verify，验证其它角色的身份。

巫师有两个特殊技能：Save 和 Poison，救人和毒人。

**夜晚共同的Action - NighttimeWhispers，**这个 Action 的设定是在夜晚的时候进行悄悄地思考和发言。大部分的Action都继承自一个 NighttimeWhispers。

**定义环境**

环境就是用来在各角色之间进行消息传递的。另外还有 round\_cnt 来控制最大交互轮数。WerewolfExtEnv 也有更新游戏和各角色状态的作用。可以大体看下环境的封装：


In [15]:
class WerewolfGame(Team):
    """Use the "software company paradigm" to hold a werewolf game"""
    env: Optional[WerewolfEnv] = None
    def __init__(self, context: Context = None, **data: Any):
        super(Team, self).__init__(**data)
        ctx = context or Context()
        if not self.env:
            self.env = WerewolfEnv(context=ctx)
        else:
            self.env.context = ctx  # The `env` object is allocated by deserialization

class WerewolfEnv(WerewolfExtEnv, Environment):
    round_cnt: int = Field(default=0)
class WerewolfExtEnv(ExtEnv):
    model_config = ConfigDict(arbitrary_types_allowed=True)
    players_state: dict[str, tuple[str, RoleState]] = Field(
        default_factory=dict, description="the player's role type and state by player_name"
    )
    round_idx: int = Field(default=0)  # the current round
    step_idx: int = Field(default=0)  # the current step of current round
    eval_step_idx: list[int] = Field(default=[])
    per_round_steps: int = Field(default=len(STEP_INSTRUCTIONS))
    # game global states
    game_setup: str = Field(default="", description="game setup including role and its num")
    special_role_players: list[str] = Field(default=[])
    winner: Optional[str] = Field(default=None)
    win_reason: Optional[str] = Field(default=None)
    witch_poison_left: int = Field(default=1, description="should be 1 or 0")
    witch_antidote_left: int = Field(default=1, description="should be 1 or 0")
    # game current round states, a round is from closing your eyes to the next time you close your eyes
    round_hunts: dict[str, str] = Field(default_factory=dict, description="nighttime wolf hunt result")
    round_votes: dict[str, str] = Field(
        default_factory=dict, description="daytime all players vote result, key=voter, value=voted one"
    )
    player_hunted: Optional[str] = Field(default=None)
    player_protected: Optional[str] = Field(default=None)
    is_hunted_player_saved: bool = Field(default=False)
    player_poisoned: Optional[str] = Field(default=None)
    player_current_dead: list[str] = Field(default=[])


NameError: name 'Optional' is not defined

**代码运行-调用metagpt/ext的角色动作**

运行过程大致为：

1. 运行代码，游戏开始，角色分配
2. 主持人走流程，黑夜守卫说话
3. 狼人杀人
4. 重复类似上述流程，直至游戏结束。

**动手操作：**
1.如果前面安装metagpt是用pip安装的，运行下面代码需要先在github获取metagpt/ext目录，因为pip的metagpt包没有ext目录：
git拉取：`git clone https://github.com/geekan/MetaGPT.git`
如果GitHub获取不成功，尝试改成第13课提供的其他代理拉取。

2.源码安装：

In [ ]:
!git clone https://gitclone.com/github.com/geekan/MetaGPT.git    
!cd MetaGPT
!pip install -e .

**创建一个werewolf.py的文件运行代码详情如下**：

In [ ]:
##运行代码详情
#导入角色和游戏相关依赖
import asyncio
import fire

from metagpt.ext.werewolf.roles import Guard, Moderator, Seer, Villager, Werewolf, Witch#守卫 主持人 先知 村民 狼人 巫师
from metagpt.ext.werewolf.roles.human_player import prepare_human_player
from metagpt.ext.werewolf.werewolf_game import WerewolfGame
from metagpt.logs import logger

#由于MetaGPT是异步框架，使用asyncio启动游戏
async def start_game(
    investment: float = 20.0,
    n_round: int = 5,#回合数，建议n_round值设置小一点
    shuffle: bool = True,
    add_human: bool = False,
    use_reflection: bool = True,
    use_experience: bool = False,
    use_memory_selection: bool = False,
    new_experience_version: str = "",
):
    game = WerewolfGame()
    #初始化游戏设置
    game_setup, players = game.env.init_game_setup(
        role_uniq_objs=[Villager, Werewolf, Guard, Seer, Witch],#设置游戏玩家职业
        num_werewolf=2,
        num_villager=2,
        shuffle=shuffle,#是否打乱职业顺序，默认打乱
        add_human=add_human,#设置真人也参与游戏
        use_reflection=use_reflection,#是否让智能体对对局信息反思，默认开启
        use_experience=use_experience,#是否让智能体根据过去行为优化自身动作，默认关闭
        use_memory_selection=use_memory_selection,
        new_experience_version=new_experience_version,
        prepare_human_player=prepare_human_player,
    )
    logger.info(f"{game_setup}")

    players = [Moderator()] + players#主持人加入游戏
    game.hire(players)
    game.invest(investment)
    game.run_project(game_setup)#主持人广播游戏情况
    await game.run(n_round=n_round)


def main(
    investment: float = 20.0,
    n_round: int = 5,#运行前建议将此处n_round修改小一点，否则对钱包不友好！！！
    shuffle: bool = True,
    add_human: bool = False,
    use_reflection: bool = True,
    use_experience: bool = False,
    use_memory_selection: bool = False,
    new_experience_version: str = "",
):
    asyncio.run(
        start_game(
            investment,
            n_round,
            shuffle,
            add_human,
            use_reflection,
            use_experience,
            use_memory_selection,
            new_experience_version,
        )
    )


if __name__ == "__main__":
    fire.Fire(main)

在命令行输入python werewolf.py ,终端会打印出对局相关消息。
**提示**
如果出现pydantic错误，降级pydantic到2.9.2